<a href="https://colab.research.google.com/github/Felipe-Moreira-Sa/agrofit/blob/main/Minutes_Metrics_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from pandas.tseries.offsets import BDay
import pandas as pd
from datetime import datetime

# Funções download_data e prepare_data permanecem as mesmas

def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    models = {
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "SVM": SVC(kernel='rbf'),
        "Naive Bayes": GaussianNB(),
        "AdaBoost": AdaBoostClassifier()
    }
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        report = classification_report(y_test, predictions, output_dict=True)
        results[name] = report
    return results

def run_stock_prediction(symbols, training_days_list, test_minutes_list):
    results = []
    end_date = datetime.now()

    for symbol in symbols:
        for training_days in training_days_list:
            data = download_data(symbol, training_days)
            if len(data) < 2:
                continue

            X, y = prepare_data(data)
            for test_minutes in test_minutes_list:
                test_size = test_minutes / len(data)
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=101)
                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)

                model_results = train_and_evaluate_models(X_train_scaled, X_test_scaled, y_train, y_test)
                for model_name, report in model_results.items():
                    results.append({
                        "Ticker": symbol,
                        "Minutos": training_days * 24 * 60,  # Convertendo dias em minutos
                        "Periodo": test_minutes,
                        "Modelo": model_name,
                        "Acurácia": report['accuracy'],
                        "Precisão": report['macro avg']['precision'],
                        "Recall": report['macro avg']['recall'],
                        "F1-Score": report['macro avg']['f1-score']
                    })

    return pd.DataFrame(results)

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [ ]:
symbols = ['PETR4.SA', 'ITUB4.SA', 'BOVA11.SA']
training_days_list = [4, 5, 6]
test_minutes_list = [30, 60, 90]

results_df = run_stock_prediction(symbols, training_days_list, test_minutes_list)
display(results_df)

NameError: name 'download_data' is not defined

In [ ]:
def convert_decimal_point_to_comma(x):
    if isinstance(x, float):
        return f"{x:.2f}".replace('.', ',')
    return x

formatted_results_df = results_df.applymap(convert_decimal_point_to_comma)
display(formatted_results_df)


In [ ]:
from google.colab import files

# Supondo que 'results_df' seja o seu DataFrame
formatted_results_df.to_csv('stock_predictions.csv', index=False)

# Baixar o arquivo CSV no seu computador
files.download('stock_predictions.csv')
